In [1]:
import os; os.environ["POLARS_VERBOSE"] = "1"
import polars as pl
import polars_ols as pls
import numpy as np

In [2]:
rng = np.random.default_rng(0)

def insert_nulls(val):
    return None if rng.random() < 0.1 else val

def _make_data(n_samples: int = 2_000, 
               n_features: int = 5,
               n_groups: int = 5,
               noise: float = 0.1,
               missing: bool = False,
               sparsity: float = 0.,
              ) -> pl.DataFrame:
    rng = np.random.default_rng(0)
    x = rng.normal(size=(n_samples, n_features))
    eps = rng.normal(size=n_samples, scale=noise)
    df = pl.DataFrame(data=x, schema=[f"x{i + 1}" for i in range(n_features)]).with_columns(
        y=pl.lit(-1 * x[:, : int(n_features * (1.0 - sparsity))].sum(1) + eps),
        group=pl.lit(rng.integers(0, n_groups, size=n_samples)),
        sample_weights=pl.lit(rng.uniform(0, 1, size=n_samples)),
    )
    if missing:
        
        df = df.with_columns(
            *(pl.col(c).map_elements(insert_nulls, return_dtype=pl.Float64) for c in df.columns)
        )
    return df

In [3]:
df = _make_data(n_samples=2_000, n_features=3, n_groups=5)

### 1. A) Basic Usage: OLS / WLS
- You can use `pls.compute_least_squares` or `least_squares.ols` from the registered namespace. They are equivalent. You need to pass (at least) a target and some features to either, see below for examples.
- Features can be specified in any of the following ways:
    - a variable number of column (string) names. E.g. `"x1", "x2", "x3"`
    - a variable number of polars expressions. E.g. `pl.col("x1"), pl.col("x2"), pl.col("x3")`)
    - a wildcard / regex multi-expression. E.g. `pl.selectors.starts_with("x"))` or `pl.col("^x.*$")`
- Simply pass an expression producing strictly positive sample weights to `sample_weights` argument to perform WLS

In [4]:
# inspect OLSKwargs for details on model specific parameters
pls.OLSKwargs?

Init signature:
pls.OLSKwargs(
    null_policy: 'NullPolicy' = 'ignore',
    alpha: 'Optional[float]' = 0.0,
    l1_ratio: 'Optional[float]' = None,
    max_iter: 'Optional[int]' = 1000,
    tol: 'Optional[float]' = 1e-05,
    positive: 'Optional[bool]' = False,
    solve_method: 'Optional[SolveMethod]' = None,
    rcond: 'Optional[float]' = None,
) -> None
Docstring:     
Specifies parameters relevant for regularized linear models: LASSO / Ridge / ElasticNet.

Attributes:
    alpha: Regularization strength. Defaults to 0.0.
    l1_ratio: Mixing parameter for ElasticNet regularization (0 for Ridge, 1 for LASSO).
        Defaults to None (equivalent to Ridge regression).
    max_iter: Maximum number of iterations. Defaults to 1000 iterations.
    tol: Tolerance for convergence criterion. Defaults to 1.e-5.
    positive: Whether to enforce non-negativity constraints on coefficients.
        Defaults to False (no constraint on coefficients).
    null_policy: Strategy for handling missing 

In [5]:
# you can use the function 'compute_least_squares'
ols_expr = pls.compute_least_squares(pl.col("y"),  # target
                          pl.selectors.starts_with("x"),  # features - can use wildcard expressions or multiple feature expressions/names
                          mode="predictions",
                          ols_kwargs=pls.OLSKwargs(null_policy="drop", solve_method="svd")
                          )

# which is equivalent to using the registered namespace
assert str(ols_expr) == str(pl.col("y").least_squares.ols(pl.selectors.starts_with("x")))

# you can make WLS by adding sample weights
wls_expr = pl.col("y").least_squares.wls("x1", "x2", "x3",  # also equivalent to pl.col("x1"), pl.col("x2"), pl.col("x3")
                                         sample_weights=pl.col("sample_weights"))

assert str(wls_expr) == str(pls.compute_least_squares("y", "x1", "x2", "x3", sample_weights="sample_weights"))

- The expressions returned are normal polars expressions. You can operate on them lazily, so for example we can compute OLS per group in parallel using `.over(...)` or multiply it by some other expression etc.

In [6]:
df.lazy().with_columns(ols_expr.over("group").alias("predictions_ols_group"),
                ols_expr.alias("predictions_ols"),
                (wls_expr * (pl.col("group") == 2)).alias("predictions_wls_masked"),
               ).collect().tail(10)

x1,x2,x3,y,group,sample_weights,predictions_ols_group,predictions_ols,predictions_wls_masked
f64,f64,f64,f64,i64,f64,f64,f64,f64
-0.583369,0.890726,0.497755,-0.70099,2,0.871927,-0.800004,-0.802822,-0.800443
0.71304,1.751887,-0.223204,-2.230821,3,0.776195,-2.241771,-2.239055,-0.0
1.098849,0.463944,-0.451817,-1.116165,2,0.01473,-1.111206,-1.110725,-1.11138
-0.485594,-0.315542,0.096269,0.866697,0,0.507687,0.70668,0.704341,0.0
0.949438,1.029228,0.318868,-2.197234,4,0.062403,-2.295554,-2.294688,-0.0
1.057735,0.268385,0.350553,-1.559323,3,0.559756,-1.67483,-1.674932,-0.0
-0.122949,2.002523,1.63392,-3.658936,3,0.585527,-3.503794,-3.506601,-0.0
-0.491295,0.870951,0.24026,-0.552929,4,0.367483,-0.617277,-0.6182,-0.0
-0.226812,0.740164,0.180547,-0.599317,4,0.119438,-0.691848,-0.692402,-0.0


- The `mode` parameter controls the type of output produced. You can choose from {`predictions`, `coefficients`, `residuals`}. It defaults to `predictions`.
- `coefficients` produces a compact struct with the names of your features as fields and estimated coefficients as values

In [7]:
df.select(pl.col("y").least_squares.ols(pl.selectors.starts_with("x"), add_intercept=True, mode="coefficients")
          .alias("coefficients"))

coefficients
struct[2]
"{-0.999496,-0.998398}"


- If in a `.over()`, `.group_by()`, or a `.with_columns()` context, the output of `mode="coefficients"` broadcasts to the shape of your data
- Computing least_squares operations in `.over()` is done in parallel in rust, so it is very efficient
- You can use `.unnest()` to unpack the coefficients to separate numeric columns

In [8]:
df_coefficients = df.select("group", pl.col("y").least_squares.ols(
   "x1", "x2", "x3", add_intercept=True, mode="coefficients").over("group")
          .alias("coefficients"))
print(df_coefficients.head())
print(df_coefficients.unnest("coefficients").head())

shape: (5, 2)
┌───────┬───────────────────────────────────┐
│ group ┆ coefficients                      │
│ ---   ┆ ---                               │
│ i64   ┆ struct[4]                         │
╞═══════╪═══════════════════════════════════╡
│ 1     ┆ {-0.993655,-0.997968,-1.006029,0… │
│ 0     ┆ {-1.002313,-1.0001,-0.993786,-0.… │
│ 4     ┆ {-1.000859,-0.998017,-0.998356,0… │
│ 3     ┆ {-0.999962,-0.999101,-0.995044,0… │
│ 3     ┆ {-0.999962,-0.999101,-0.995044,0… │
└───────┴───────────────────────────────────┘
shape: (5, 5)
┌───────┬───────────┬───────────┬───────────┬───────────┐
│ group ┆ x1        ┆ x2        ┆ x3        ┆ const     │
│ ---   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ i64   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞═══════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 1     ┆ -0.993655 ┆ -0.997968 ┆ -1.006029 ┆ 0.003129  │
│ 0     ┆ -1.002313 ┆ -1.0001   ┆ -0.993786 ┆ -0.006384 │
│ 4     ┆ -1.000859 ┆ -0.998017 ┆ -0.998356 ┆ 0.004797  │
│ 

### 1. B) Null Policy & Solve Methods

- Most models expose a choice over multiple algorithms ("solve_method") and multiple strategies in handling missing data ("null_policy")
- The next section explains how some of these options work

#### Make some data with random nulls

In [9]:
df_missing = _make_data(missing=True)

In [10]:
df_missing

x1,x2,x3,x4,x5,y,group,sample_weights
f64,f64,f64,f64,f64,f64,f64,f64
0.12573,-0.132105,0.640423,0.1049,-0.535669,-0.154338,2.0,0.313882
0.361595,1.304,0.947081,-0.703735,null,-0.777174,1.0,0.215647
null,0.041326,-2.325031,null,-1.245911,4.260299,0.0,0.975329
null,-0.544259,-0.3163,0.411631,1.042513,0.093387,1.0,0.349839
-0.128535,1.366463,-0.665195,0.35151,null,-1.659732,1.0,0.180044
…,…,…,…,…,…,…,…
null,0.433351,null,0.659305,0.346436,null,2.0,0.921171
0.73432,-0.562784,0.642383,-0.328101,-0.704787,0.239824,1.0,0.930792
-0.880797,-0.497476,-0.464837,1.577698,0.045559,0.249027,1.0,0.120942


In [11]:
pls.NullPolicy

typing.Literal['zero', 'drop', 'ignore', 'drop_zero', 'drop_y_zero_x', 'drop_window']

#### i- null_policy: "zero": simply equivalent to zero filling prior to fitting

In [12]:
pred_zero = df_missing.select(
    pls.compute_least_squares("y", pl.selectors.starts_with("x"), 
                              mode="predictions",
                              ols_kwargs=pls.OLSKwargs(null_policy="zero"))
)

expected = df_missing.fill_null(0.).select(pl.col("y").least_squares.ols(pl.selectors.starts_with("x")))

assert np.allclose(pred_zero, expected)


# notice that residuals, however, will keep the nulls in the original targets (resid := y - y_hat)
print(df_missing.select(pl.col("y").least_squares.ols(
    pl.selectors.starts_with("x"),
    mode="residuals",
    null_policy="zero",
)).tail())

shape: (5, 1)
┌───────────┐
│ y         │
│ ---       │
│ f64       │
╞═══════════╡
│ null      │
│ 0.081255  │
│ 0.026447  │
│ -0.377051 │
│ 0.409127  │
└───────────┘


#### ii- null_policy: "drop": is equivalent to dropping rows with nulls in targets or features prior to fitting
- but then 'broadcasting' predictions to original data shape

In [13]:
coef_drop = df_missing.select(
    pl.col("y").least_squares.ols("x1", "x2", 
                              mode="coefficients",
                              null_policy="drop")
)


expected = df_missing.drop_nulls(subset=["y", "x1", "x2"]
                                ).select(pl.col("y").least_squares.ols("x1", "x2", mode="coefficients"))

assert np.allclose(coef_drop.unnest("coefficients"), expected.unnest("coefficients"))

dataframe filtered


#### iii- null_policy: "drop_y_zero_x": is equivalent to masking only rows with nulls in targets
- and then zeroing out any remainining nulls in features prior to fitting and prediction
- but then 'broadcasting' predictions to original data shape

In [14]:
coef_drop_y = df_missing.select(
    pl.col("y").least_squares.ols("x1", "x2", 
                              mode="coefficients",
                              null_policy="drop_y_zero_x")
)

expected = (
    df_missing.drop_nulls(subset=["y"]).fill_null(0.)
    .select(pl.col("y").least_squares.ols("x1", "x2", mode="coefficients"))
)

assert np.allclose(coef_drop_y.unnest("coefficients"), expected.unnest("coefficients"))

dataframe filtered


In [15]:
pls.SolveMethod

typing.Literal['qr', 'svd', 'chol', 'lu', 'cd', 'cd_active_set']

#### Overview of Solve Methods: 
- *"qr"*: Uses QR decomposition with partial pivoting to decompose 'X' into a product of a unitary and upper trapezoidal matrices. The decomposition is then used to solve least squares equations. See [here](https://docs.rs/faer-qr/latest/faer_qr/)
- *"svd"*: Uses Singular Value Decomposition (or, if on linux or macos, LAPACK accelerated divide and conquer SVD) to identify the minimum norm solution for least squares problem. This is usually the best approach for singular / multicollinear problems. See [here](https://netlib.org/lapack/explore-html-3.6.1/d7/d3b/group__double_g_esolve_ga479cdaa0d257e4e42f2fb5dcc30111b1.html).
- *"chol"*: Uses a simple cholesky decomposition of XTX and directly solves the normal equations XTX vs XTY. This approach is often the fastest but assumes well behaved data (e.g. full rank). This may be chosen as a suitable method to solve ridge problems. 
- *"lu"*: Uses LU decomposition with pivoting to directly solve the normal equations
- *"cd"*: short for 'Coordinate Descent'. This approach is usesd by more complex models (which don't admit analytic solutions) and either use L1 penalty or impose non-negativity constraint on coefficients. It works by iteratively updating coefficients, one feature at a time, an approach which is very effective for elastic net type problems. See implementation [here](https://github.com/azmyrajab/polars_ols/blob/ce499a7e2f450378441421bcb1512b57a816af2c/src/least_squares.rs#L328).
- *"cd_active_set"*: For problems with large number of features and high degree of expected sparsity, we know that most coefficients will be zero. In such problems, cyclic coordinate descent can be accelerated by 'turning off' coefficients which have converged to zero and focusing remaining iterations on non-zero features.

### Example: solve_method: "svd" to recover minimum norm solution in presence of multi-collinearity

In [16]:
df_collinear = (
    df
    .select("x1", "x2", "x3")
    .with_columns(pl.col("x2").alias("x3"))  # I add a totally collinear feature x3 which is a copy of x2
    .with_columns(pl.sum_horizontal(pl.col("^x.*$")).alias("y"))  # y is the sum of x1, .., x3 
)

#### QR identifies a 'correct', but not minimum norm, solution (out of the infinite ones)

In [17]:
coef_qr = df_collinear.select(pl.col("y").least_squares.ols("x1", "x2", "x3", solve_method="qr", mode="coefficients"))
print("norm=",np.linalg.norm(coef_qr.unnest("coefficients")))
print(coef_qr)

norm= 2.23606797749979
shape: (1, 1)
┌────────────────┐
│ coefficients   │
│ ---            │
│ struct[3]      │
╞════════════════╡
│ {1.0,2.0,-0.0} │
└────────────────┘


#### Cholesky fails to compute

In [18]:
df_collinear.select(pl.col("y").least_squares.ols("x1", "x2", "x3", solve_method="chol", mode="coefficients"))

Cholesky decomposition failed, falling back to LU w/ pivoting


coefficients
struct[3]
"{null,null,null}"


#### SVD computes the minimum norm solution [1, 1, 1]

In [19]:
df_collinear.select(pl.col("y").least_squares.ols("x1", "x2", "x3", solve_method="svd", mode="coefficients"))

coefficients
struct[3]
"{1.0,1.0,1.0}"


#### It matches np.linalg.lstsq which uses the same algorithm

In [20]:
np.linalg.lstsq(df_collinear.select("x1", "x2", "x3"), df_collinear["y"], rcond=None)[0]

array([1., 1., 1.])

### 2. Regularized Models
- Ridge `least_squares.ridge`, Lasso `least_squares.lasso`, Elastic Net `least_squares.lasso` with optional non-negative constraint are implemented
- Apart from ridge, which is solved in closed form, the rust implementation for regularized models is cyclic coordinate descent with a soft thresholding function that supports an arbitrary combination of L1 / L2 penalties and non-negative constraint.
- `sample_weights` and `mode` are general parameters applicable to all models supported by this package

Parameters specific to regularized models are contained in `OLSKwargs`:
- alpha: scalar representing L1 or L2 penalty strength.
- l1_ratio: mixing parameter for ElasticNet regularization (0 for Ridge, 1 for LASSO).
- max_iter: maximum number of coordinate descent iterations
- tol: tolerance for convergence criterion
- positive: boolean enforcing non-negativity constraints on coefficients

In [21]:
elastic_net_expr = pl.col("y").least_squares.elastic_net(pl.col("x1"), pl.col("x2"), pl.col("x3"),
                                                         alpha=0.0001,
                                                         l1_ratio=0.5,
                                                         positive=True,
                                                         mode="coefficients",
                                                         ).alias("coef_enet_non_negative")

ridge_expr = pl.col("y").least_squares.ridge(pl.col("x1"), pl.col("x2"), pl.col("x3"),
                                             alpha=100.0, 
                                             sample_weights=pl.col("sample_weights"),
                                             mode="coefficients").alias("coef_ridge")

df.select(elastic_net_expr, ridge_expr)

coef_enet_non_negative,coef_ridge
struct[3],struct[3]
"{0.0,0.0,0.0}","{-0.912021,-0.898583,-0.908957}"


### Example: Elastic Net w/ large number of predictors
- most of them are sparse / zero

In [22]:
from sklearn.linear_model import ElasticNet

In [46]:
df_wide = _make_data(n_samples=1_000, n_features=1_000, sparsity=0.9)
features = [pl.col(f"x{i+1}") for i in range(1_000)]

In [52]:
# implementation matches sklearn 
coef = df_wide.select(pl.col("y").least_squares.elastic_net(
    *features,
    l1_ratio=0.5,
    alpha=0.1,
    mode="coefficients",
    solve_method="cd",
    max_iter=1_000,
    tol=0.0001,
)).unnest("coefficients")

mdl = ElasticNet(l1_ratio=0.5, max_iter=1_000,  tol=0.0001, fit_intercept=False, alpha=0.1)
x, y = df_wide.select(*features).to_numpy(), df_wide.select("y").to_numpy()
mdl.fit(x, y)

assert np.allclose(mdl.coef_, coef.to_numpy(), rtol=1.e-4, atol=1.e-4)

In [53]:
%%timeit -n 20
df_wide.with_columns(pl.col("y").least_squares.elastic_net(
    *features,
    l1_ratio=0.5,
    alpha=0.1,
    mode="predictions",
    solve_method="cd",
    max_iter=1_000,
    tol=0.0001,
).alias("predictions"))

45.7 ms ± 3.26 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [54]:
%%timeit -n 20
df_wide.with_columns(pl.col("y").least_squares.elastic_net(
    *features,
    l1_ratio=0.5,
    alpha=0.1,
    mode="predictions",
    solve_method="cd_active_set",  # can get a meaningful speed up by using active set
    max_iter=1_000,
    tol=0.0001,
).alias("predictions"))

21.2 ms ± 4.02 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)


### 3. Formula API

- For those who like specifying models in patsy formula syntax, that is also supported
- You can either use the `least_squares_from_formula` module level public function or `least_squares.from_formula` from registed namespace
- It tries to be clever and maps to the correct underlying implementation based on the model specific parameters you specify

In [55]:
# compute the residuals in two equivalent ways
df.select(
    # "x2:x3" denotes multiplicative interaction, "-1" dentotes no intercept
    pls.compute_least_squares_from_formula("y ~ x1 + x2:x3 -1", mode="residuals").alias("residuals_1"), 
    (pl.col("y") - pl.col("y").least_squares.from_formula("x1 + x2:x3 -1", mode="predictions")).alias("residuals_2"),
).corr()

residuals_1,residuals_2
f64,f64
1.0,1.0
1.0,1.0


In [56]:
nnls_formula_expr = pl.col("y").least_squares.from_formula("x1 + x2 + x3",
                                       alpha=0.0001,
                                       positive=True,
                                       )  # knows to use the coordinate descent implementation because of non-negativity


ridge_formula_expr = pl.col("y").least_squares.from_formula("x1 + x2 + x3",
                                       alpha=0.0001,
                                       sample_weights=pl.col("sample_weights"),
                                       )  # knows that it needs to use closed form ridge w/ sample weighting

### 4. Dynamic Regression Models

- Consider the situation where you want to compute coefficients in an expanding or rolling window manner
    - naively, you could manually re-compute standard OLS function over consecutive windows (e.g. `.rolling(...).agg(...)`)
    - ... but that would be wasteful: (X.T X) and (X.T Y) are only changing by one row (in case of expanding) or two rows (in case of rolling, an addition and a subtraction)
- This extension package provides rust implementations `.least_squares.{rolling_ols, expanding_ols, rls}` which efficiently update coefficients as new samples are observed
- The key idea is to make use of Sherman-Morrison or Woodbury Identity to recursively update summary statistics or coefficient vectors
- Formula API is also supported and the correct implementation is chosen based on parameters provided

In [57]:
df.select(
    pl.col("y").least_squares.rolling_ols("x1", "x2", "x3", 
                                           window_size=252, 
                                           min_periods=5, 
                                           alpha=0.0001,  
                                           mode="coefficients"
                                          ).over("group").alias("rolling_ridge_coef"),
    pl.col("y").least_squares.rls(
        "x1", "x2", "x3",
        half_life=21.0, # exponential memory proportional to a half-life of 21 samples
        initial_state_mean=[-1.0, -1.0, -1.0],  # prior mean for initial coefficients
        initial_state_covariance=10.0,  # inversely proportional to L2 prior towards prior mean
        mode="coefficients",
    ).over("group").alias("recursive_least_squares_coef"),
    pl.col("y").least_squares.expanding_ols(pl.col("x1"), pl.col("x2"), pl.col("x3"), 
                                           mode="predictions").alias("expanding_ols_pred"),
)

rolling_ridge_coef,recursive_least_squares_coef,expanding_ols_pred
struct[3],struct[3],f64
"{null,null,null}","{-1.031467,-0.966938,-1.160281}",-0.627675
"{null,null,null}","{-1.013228,-0.932454,-1.045596}",-0.127212
"{null,null,null}","{-1.003344,-1.002429,-0.998195}",-1.48872
"{null,null,null}","{-1.053291,-1.026248,-0.99826}",1.852231
"{null,null,null}","{-1.059251,-1.017933,-1.014118}",3.941405
…,…,…
"{-0.995486,-1.003153,-0.998155}","{-0.987406,-1.004047,-1.004306}",-1.674772
"{-0.995348,-1.004411,-0.99898}","{-0.985392,-1.011036,-1.00789}",-3.506569
"{-1.001142,-1.002267,-1.002736}","{-1.001857,-0.980484,-1.007321}",-0.618197


### 5. Out Of Sample Prediction

- If you want to fit on some data then predict on test data, you can do so with `least_squares.predict(...)`

In [58]:
# make some random training data
df_train = _make_data(n_groups=1)

# fit coefficients
df_coefficients = (
    df.lazy()
    .select(
        "group",
        pl.col("y")
        .least_squares.ols(pl.col("x1"), pl.col("x2"), mode="coefficients")
        .over("group").alias("coefficients"),
    )
    .unique()
)

df_coefficients.collect()

group,coefficients
i64,struct[2]
3,"{-1.074243,-1.050159}"
0,"{-1.028893,-1.004733}"
2,"{-0.948578,-0.917406}"
4,"{-1.037308,-0.981468}"
1,"{-1.094156,-0.946028}"


In [59]:
# make some test data
df_test = _make_data(n_groups=1)

# 1) join on group or common index columns etc.
# 2) compute predictions by calling least_squares.predict(coefficient_column, *feature_columns)
predictions = (
    df_test.lazy()
    .join(df_coefficients, on="group")
    .select(
        "group",
        pl.col("coefficients").least_squares.predict(
            pl.col("x1"), pl.col("x2"), name="predictions_test"
        )
    )
    .collect()
)

predictions.head()

join parallel: true
INNER join dataframes finished


group,predictions_test
i64,f64
0,0.003367
0,-1.682214
0,0.599761
0,1.30026
0,-1.240682
